# Tratamento de Lista Geral de Leads

### Objetivo: Esse pipeline tem como objetivo separar a lista de Leads disponível para disparos e consumo

In [3]:
# Importações básicas
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import os
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Adiciona src ao path
sys.path.append('../src')

#

# Utilitários de dados
from data_utils import (
    load_raw_data,
    save_processed_data,
    remove_duplicates,
    handle_missing_values,
    detect_outliers,
    normalize_column,
    process_phone_string,
    process_phone_number,
    clean_and_lower_column,
    flatten_list_to_df,
    remove_buyers_from_dataframe
)

CRONOGRAMA_SUBDOMAIN = 'cronogramadosfluentes-xwamel'

# Utilitários SQL
from sql_utils import DatabaseConnection as Dbc, load_query_from_file

# Utilitários de visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Utilitários de API
from api_utils import (
    make_request,
    get_json,
    post_json,
    paginated_request,
    response_to_dataframe
)

# utilitários hotmart
from hotmart_utils import Hotmart

# utilitários tmb
from tmb_utils import TMB   

# Configurações pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Load Database Driver
db = Dbc()

# Inicializar API Hotmart
hotmart = Hotmart()

# Inicializar API TMB
tmb = TMB()

print('✓ Importações concluídas com sucesso!')

✓ Importações concluídas com sucesso!


## Load Leads Dataframe

In [4]:
CLASS_DATE = "2025-12-30 22:07:00+00"
df = db.execute_query_from_file("select_as_checkins", params={"class_date": CLASS_DATE})

## Load Emails, Phones and names

In [5]:
emails = db.execute_query_from_file("select_all_emails")
phones = db.execute_query_from_file("select_leads_last_phone")
names = db.execute_query_from_file("select_all_names")

## Merging DF

In [ ]:
df_email = pd.merge(df, emails, on="email", how="left")

df_email = df_email[df_email["lead_id"].notna()]

df_email_phone = pd.merge(df_email, phones, on="lead_id", how="left")

df_email_phone_name = pd.merge(df_email_phone, names, on="lead_id", how="left")

# Drop columns: 'reprocessed', 'created_at_x' and 12 other columns
df_email_phone_name = df_email_phone_name.drop(columns=['reprocessed', 'created_at_x', 'expiration_date', 'is_active', 'id_x', 'active_id', 'created_at_y', 'checked', 'id_y', 'raw_phone_input', 'phone_type', 'ddd', 'ddi', 'is_valid'])

# Rename column 'formatted_phone' to 'phone'
df_email_phone_name = df_email_phone_name.rename(columns={'formatted_phone': 'phone'})

## Filter Buyers from Dataframe
df = remove_buyers_from_dataframe(df_email_phone_name)

✓ Removidos 123 compradores de 2,329 registros (5.3% filtrado)


KeyError: 'last_conversion_date'